In [5]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Hourly
import numpy as np
import pandas as pd

In [6]:
data = pd.read_csv("../processed/processed-shutoffs.csv")
data.head()

,circuit_name,deenergize_time,restoration_time,time_out_min,key_communities,hftd_tier,total_affected,residential_affected,zip_code,longitude,latitude,substn_present,total_population
0,APPLE HILL-1103,2018-10-14 21:00:00,2018-10-15 16:29:00.000000000,1169.0,"Camino, Placerville",3.0,1809.0,1488.0,95709,-120.677165,38.739683,True,5634.0
1,APPLE HILL-1104,2018-10-14 21:17:00,2018-10-15 18:37:00.000000000,1280.0,"Camino, Placerville, Pollock Pines,",3.0,2261.0,2043.0,95709,-120.677165,38.739683,True,5634.0
2,APPLE HILL-2102,2018-10-14 21:05:00,2018-10-16 18:04:00.000000000,2699.0,"Camino, Grizzly Flats, Mount Aukum, Placervill...",3.0,4489.0,4013.0,95709,-120.677165,38.739683,True,5634.0
3,BONNIE NOOK-1102,2018-10-14 21:54:00,2018-10-15 18:04:00.000000000,1210.0,Alta,3.0,533.0,454.0,95701,-120.809292,39.214708,True,1064.0
4,CALISTOGA-1101,2018-10-14 20:34:00,2018-10-15 21:27:00.000000000,1493.0,Calistoga,3.0,1596.0,1216.0,94515,-122.578610,38.592088,True,7525.0


In [8]:
data["denergize_time"] = pd.to_datetime(data["deenergize_time"])
data["restoration_time"] = pd.to_datetime(data["restoration_time"])
data["deenergize_date"] = data["denergize_time"].dt.date
data["restoration_date"] = data["restoration_time"].dt.date


In [9]:
missing = []
# hourly = {}
daily = {}
for i in range(data.shape[0]):
    point_1 = Point(data.loc[i, "latitude"], data.loc[i, "longitude"])
    start = data.loc[i, "denergize_time"] - timedelta(days=5)
    end = data.loc[i, "restoration_time"]
    # weather = Hourly(point_1, start, end)
    weather = Daily(point_1, start, end)
    weather = weather.fetch()
    # if weather.shape[0] == 0:
    #     weather = Daily(point_1, start, end)
    #     weather = weather.fetch()
    if weather.shape[0] == 0:
        missing.append(i)
    # else:
    #     daily[i] = weather
    else:
        daily[i] = weather

In [10]:
daily

{0:             tavg  tmin  tmax  prcp  snow  wdir  wspd  wpgt  pres  tsun
 time                                                                  
 2018-10-10  16.6  13.0  21.0   NaN   NaN   NaN   8.5   NaN   NaN   NaN
 2018-10-11  14.6  10.0  19.0   NaN   NaN   NaN   6.4   NaN   NaN   NaN
 2018-10-12  18.8  14.0  23.0   NaN   NaN   NaN  10.7   NaN   NaN   NaN
 2018-10-13  18.7  14.0  23.0   NaN   NaN   NaN   9.2   NaN   NaN   NaN,
 1:             tavg  tmin  tmax  prcp  snow  wdir  wspd  wpgt  pres  tsun
 time                                                                  
 2018-10-10  16.6  13.0  21.0   NaN   NaN   NaN   8.5   NaN   NaN   NaN
 2018-10-11  14.6  10.0  19.0   NaN   NaN   NaN   6.4   NaN   NaN   NaN
 2018-10-12  18.8  14.0  23.0   NaN   NaN   NaN  10.7   NaN   NaN   NaN
 2018-10-13  18.7  14.0  23.0   NaN   NaN   NaN   9.2   NaN   NaN   NaN,
 2:             tavg  tmin  tmax  prcp  snow  wdir  wspd  wpgt  pres  tsun
 time                                                

In [4]:
pd.read_csv("raw/weather_ALHAMBRA-1105_2019-10-26_hourly.csv")

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,2019-10-24 22:00:00,33.9,-6.7,7.0,0.0,NaN,110.0,14.8,NaN,1015.1,NaN,1.0
1,2019-10-24 23:00:00,33.3,-5.4,8.0,0.0,NaN,110.0,18.4,NaN,1015.2,NaN,1.0
2,2019-10-25 00:00:00,31.1,-5.5,9.0,0.0,NaN,110.0,14.8,NaN,1015.2,NaN,2.0
3,2019-10-25 01:00:00,28.3,-3.8,12.0,NaN,NaN,100.0,14.8,NaN,1015.1,NaN,1.0
4,2019-10-25 02:00:00,22.2,-2.5,19.0,NaN,NaN,50.0,0.0,NaN,1015.5,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
90,2019-10-28 16:00:00,11.1,-8.7,24.0,0.0,NaN,140.0,0.0,NaN,1017.6,NaN,0.0
91,2019-10-28 17:00:00,15.6,-7.3,20.0,0.0,NaN,240.0,7.6,NaN,1018.1,NaN,0.0
92,2019-10-28 18:00:00,16.7,-7.8,18.0,0.0,NaN,210.0,9.4,NaN,1017.9,NaN,0.0
93,2019-10-28 19:00:00,18.3,-8.8,15.0,0.0,NaN,190.0,13.0,NaN,1017.5,NaN,1.0


In [ ]:
missing = pd.DataFrame(missing)
missing = missing.rename(columns={0: "index"})
missing["Circuit Name"] = data.iloc[missing["index"], :].reset_index()[
    "circuit_name"]
missing.to_csv("missing_weather.csv")


In [17]:
len(missing) / data.shape[0]

0.1665886757136172

In [13]:
for i in list(daily.keys()):
    shutoff = data.iloc[i, :]
    filename = f"raw/daily/weather_{shutoff['circuit_name']}_{shutoff['deenergize_date']}.csv"
    weather = daily[i]
    weather.to_csv(filename)
    

In [51]:
# for i in list(hourly.keys()):
#     shutoff = data.iloc[i, :]
#     filename = f"raw/hourly/weather_{shutoff['circuit_name']}_{shutoff['deenergize_date']}_hourly.csv"
#     weather = hourly[i]
#     weather.to_csv(filename)
